In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

spark = SparkSession.builder \
    .appName("Localspark") \
    .master("local[*]") \
    .config("spark.jars.packages",
        "com.microsoft.sqlserver:mssql-jdbc:12.6.1.jre11") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [2]:
# Load variables from .env into os.environ
load_dotenv("env")
server_name = os.environ.get("SERVERNAME")
database_name = os.environ.get("DATABASENAME")
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

In [7]:
# Standard JDBC URL format for SQL Server
jdbc_url = f"jdbc:sqlserver://{server_name}:1433;databaseName={database_name}"

#table name to read
table_name = "BIB.Autor"

In [8]:
try:
    jdbcDF = spark.read \
        .format("jdbc") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .load()

    # Display schema and a few rows if successful
    jdbcDF.printSchema()
    jdbcDF.show(5)

except Exception as e:
    print("JDBC read failed!")
    print("Error:", e)

root
 |-- ID_Autor: integer (nullable = true)
 |-- Vorname_1: string (nullable = true)
 |-- Vorname_2: string (nullable = true)
 |-- Nachname: string (nullable = true)

+--------+---------+---------+--------+
|ID_Autor|Vorname_1|Vorname_2|Nachname|
+--------+---------+---------+--------+
|       1|      Abe|     NULL|    Kobo|
|       2|   Chinua|     NULL|  Achebe|
|       3|  Theodor|       W.|  Adorno|
|       4|     NULL|     NULL|    Äsop|
|       5|   Samuel|     NULL|   Agnon|
+--------+---------+---------+--------+
only showing top 5 rows



In [14]:
autor_df = jdbcDF.select("*")

In [12]:
autor_df.show()

+--------+---------+---------+--------------+
|ID_Autor|Vorname_1|Vorname_2|      Nachname|
+--------+---------+---------+--------------+
|       1|      Abe|     NULL|          Kobo|
|       2|   Chinua|     NULL|        Achebe|
|       3|  Theodor|       W.|        Adorno|
|       4|     NULL|     NULL|          Äsop|
|       5|   Samuel|     NULL|         Agnon|
|       6|     Ilse|     NULL|     Aichinger|
|       7|Tschingis|     NULL|      Aitmatow|
|       8|     NULL|     NULL|Alain-Fournier|
|       9|    Mateo|     NULL|        Alemán|
|      10|   Nelson|     NULL|        Algren|
|      11|   Isabel|     NULL|       Allende|
|      12|    Jorge|     NULL|         Amado|
|      13|   Ingvar|     NULL|    Ambjørnsen|
|      14|     Eric|     NULL|        Ambler|
|      15|     Jean|     NULL|         Améry|
|      16|  Günther|     NULL|        Anders|
|      17|   Alfred|     NULL|      Andersch|
|      18|     Hans|Christian|      Andersen|
|      19|   Martin| Andersen|    